In [1]:
import seaborn as sns

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [4]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [5]:
df.duplicated().sum()

1

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [8]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [9]:
df['smoker'].unique()

['No', 'Yes']
Categories (2, object): ['Yes', 'No']

In [10]:
df.duplicated().sum()

0

In [11]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [12]:
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df[['time']])

c:\Users\shekh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [14]:
df['time'].value_counts()

time
0    176
1     67
Name: count, dtype: int64

In [15]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [16]:
x=df.drop(labels=['time'],axis=1)
y = df['time']

In [17]:
x

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [18]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 243, dtype: int32

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=42)

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [31]:
categorical_feature = ['sex','smoker','day']
numerical_feature = ['total_bill','tip','size']

In [39]:
##Feature Engineering automation

#num pipeline for feature engineering
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)
#Categorical pipeline for feature engineering
categorical_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('oneHotEncoder',OneHotEncoder())
    ]
)
preprocessor = ColumnTransformer(
    [
        ('num_pipeline',num_pipeline,numerical_feature),
        ('categorical_pipeline',categorical_pipeline,categorical_feature)
    ]
)


In [40]:
x_train = preprocessor.fit_transform(x_train)


In [41]:
x_test = preprocessor.transform(x_test)

In [43]:
x_train

array([[-0.31281158,  0.34930121, -0.63021145, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.48521345, -0.75494133, -0.63021145, ...,  1.        ,
         0.        ,  0.        ],
       [-0.76227396, -0.3192538 , -0.63021145, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.63826695, -1.52866229, -0.63021145, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.6730783 ,  0.38686048, -0.63021145, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.7818286 , -0.40188419,  0.42376287, ...,  1.        ,
         0.        ,  0.        ]])

In [73]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


models = {
    'RandomForestClassifier':RandomForestClassifier(),
    'LogisticRegression':LogisticRegression(),
    'DecisionTreeClassifier':DecisionTreeClassifier()
}

In [74]:
from sklearn.metrics import accuracy_score

In [75]:
def evaluate_model(x_train,x_test,y_train,y_test,models):
    report = {}

    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(x_train,y_train)
        y_pred = model.predict(x_test)

        score = accuracy_score(y_test,y_pred)

        report[list(models.keys())[i]] = score

    return report





In [77]:
score = evaluate_model(x_train,x_test,y_train,y_test,models)

In [78]:
score

{'RandomForestClassifier': 0.9795918367346939,
 'LogisticRegression': 1.0,
 'DecisionTreeClassifier': 0.9795918367346939}

In [83]:
max(sorted(score.values(),reverse=True))

1.0